In [2]:
import glob
import pandas
import numpy

from plots import plot_boxes

In [3]:
target_dir = 'results/'

data_load = None
for file_name in glob.glob(target_dir + '*'):
    print('Loading...\t' + file_name)
    if data_load is None:
        data_load = pandas.read_pickle(file_name)
    else:
        data_load = pandas.concat([data_load,
                                   pandas.read_pickle(file_name)], 
                                  sort=False)

Loading...	results/gaussian_covariance_100_20190227
Loading...	results/baseline_identity_framingham_2_20190227
Loading...	results/gaussian_identity_framingham_2_20190227


In [21]:
print(data_load['mechanism'].value_counts())
print(data_load.columns.values)
data_load[data_load.mechanism == 'baseline']


gaussian    102
baseline      2
Name: mechanism, dtype: int64
['mechanism' 'query' 'iteration' 'metric' 'result' 'mechanism runtime (s)'
 'total runtime (s)']


,mechanism,query,iteration,metric,result,mechanism runtime (s),total runtime (s)
0,baseline,identity,1,rmse,0.059814,0.0,34.152287
0,baseline,identity,2,rmse,0.062156,0.0,34.953787


In [20]:
data_load[data_load.mechanism == 'gaussian']

,mechanism,query,iteration,metric,result,mechanism runtime (s),total runtime (s)
0,gaussian,covariance,1,principle component RSS,0.398067,0.009271,0.013078
0,gaussian,covariance,2,principle component RSS,0.402775,0.008868,0.012632
0,gaussian,covariance,3,principle component RSS,0.395578,0.008717,0.012295
0,gaussian,covariance,4,principle component RSS,0.403719,0.008641,0.012411
0,gaussian,covariance,5,principle component RSS,0.393647,0.010706,0.014286
0,gaussian,covariance,6,principle component RSS,0.394603,0.010799,0.014289
0,gaussian,covariance,7,principle component RSS,0.401758,0.008475,0.013420
0,gaussian,covariance,8,principle component RSS,0.401552,0.008689,0.012193
0,gaussian,covariance,9,principle component RSS,0.391818,0.008867,0.012480
0,gaussian,covariance,10,principle component RSS,0.403471,0.009964,0.013592


# Compare Differentially Private (DP) Sample Covarince / SVD methods

In [25]:
sample_cov_data = data_load[(data_load.query == 'covariance') & (data_load.metric == 'principle component RSS')]

In [29]:
# Collect data and labels
boxplot_labels = [ sample_size for sample_size in sample_cov_data['sample size'].unique()]
boxplot_data = [ sample_cov_data[sample_cov_data['sample size'] == sample_size].value_counts() ] 
                for sample_size in sample_cov_data['sample size'].unique()]

['gaussian', 'baseline']

In [ ]:
# Boxplot of experiment RMSE results 
boxplot_args = dict(notch=False,bootstrap=1000)

plot, boxplot = plot_boxes(
        title='Comparing Captured Variance of DP\nSample Covariance Methods',
        xlabel='Sample Size',
        ylabel='Principal Component',
        xplot_labels=e1_rmse_labels,
        plot_data=e1_rmse_data,
        xplot_label_args=dict(rotation=45, fontsize=8),
        **boxplot_args)